# chapter 2. Machine Learning Starting with Scikit-Learn

## Model Selection Module 

### 학습 / 테스트 데이터 세트 분리 - train_test_split()

사이킷런의 **model_selection 모듈**은 학습 데이터와 테스트 데이터 세트를 분리하거나 교차 검증 분할 및 평가, 그리고 Estimator의 하이퍼 파라미터를 튜닝하기 위한 다양한 함수와 클래스를 제공한다. <br>

**학습 / 테스트 데이터 세트 분리 - train_test_split()** <br>

학습 데이터 세트로만 학습하고 예측하면, 이미 학습한 데이터 세트를 기반으로 예측하는 것이기 때문에 정확도가 100%이 나오는 상황이 발생한다. <br> 

In [3]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

iris = load_iris()
dt_clf = DecisionTreeClassifier()
train_data = iris.data
train_label = iris.target
dt_clf.fit(train_data, train_label)

# 학습 데이터 세트로 예측 수행
pred = dt_clf.predict(train_data)
print('예측 정확도 : ', accuracy_score(train_label, pred))

예측 정확도 :  1.0


사이킷런의 train_test_split()을 통해 **원본 데이터 세트에서 학습 및 테스트 데이터 세트를 쉽게 분리**할 수 있다. <br>

1. sklearn.model_selection 모듈에서 train_test_split()을 로드한다. <br>
2. train_test_split()은 첫번째 파라미터로 피처 데이터 세트, 두번째 파라미터로 레이블 데이터 세트를 입력받는다. <br>
3. 선택적으로 다음 파라미터를 입력받는다. 

|용어|설명|
|:------:|:---:|
|test_size|전체 데이터에서 테스트 데이터 세트 크기를 얼마로 샘플링 할 것인가를 결정한다. <br> 디폴트는 0.25, 즉 25%이다.|
|train_size|전체 데이터에서 학습용 데이터 세트 크기를 얼마로 샘플링 할 것인가를 결정한다. <br> test_size parameter를 통상적으로 사용하기 때문에 train_size는 잘 사용하지 않는다.|
|shuffle|데이터를 분리하기 전에 데이터를 미리 섞을지 결정한다. <br> 디폴트는 True이다. 데이터를 분산시켜서 좀 더 효율적인 학습 및 테스트 데이터 세트를 만드는데 사용한다.|
|random_state|random_state는 호출할 때마다 동일한 학습 / 테스트용 데이터 세트를 생성하기 위해 주어지는 난수 값이다. <br> train_test_split()은 호출 시 무작위로 데이터를 분리하므로 ramdon_state를 지정하지 않으면 수행할 때마다 다른 학습 / 테스트용 데이터를 생성한다.|

<br>

**train_test_split()의 반환 값은 튜플 형태**이다. <br>
순차적으로 학습용 데이터의 피처 데이터 세트, 테스트용 데이터의 피처 데이터 세트, 학습용 데이터의 레이블 데이터 세트, 테스트용 데이터의 레이블 데이터 세트가 반환된다. 

In [5]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

dt_clf = DecisionTreeClassifier()
iris_data = load_iris()

X_train, X_test, Y_train, Y_test = train_test_split(iris_data.data, iris_data.target, 
                                                   test_size = 0.3, random_state = 121)

In [6]:
dt_clf.fit(X_train, Y_train)
pred = dt_clf.predict(X_test)
print('예측 정확도 : {0:.4f}'.format(accuracy_score(Y_test, pred)))

예측 정확도 : 0.9556


### 교차 검증

알고리즘을 학습시키는 학습 데이터와 이에 대한 예측 성능을 평가하기 위한 별도의 테스트용 데이터를 필요로 한다. <br>
이 방법은 **과적합(Overfitting)** 에 취약한 약점을 가진다. <br>
과적합은 **모델이 학습 데이터에만 과도하게 최적화**되어, **실제 예측**을 다른 데이터로 수행할 경우에는 **예측 성능이 과도하게 떨어지는 것**을 말한다. <br>
**테스트 데이터에만 최적의 성능을 발휘**할 수 있도록 편향되게 모델을 유도하는 경향인 것이다. <br>

교차 검증을 사용하게 되면 이 문제점을 개선할 수 있다. <br>
**교차 검증**은 **데이터 편중을 막기 위해서 별도의 여러 세트로 구성된 학습 데이터 세트와 검증 데이터 세트에서 학습과 평가를 수행**하는 것이다. <br>
교차 검증을 사용하게 되면 모델 최적화를 더 쉽게 할 수 있다. <br>
대부분의 **ML 모델의 성능 평가는 교차 검증 기반으로 1차 평가를 한 뒤 최종적으로 테스트 데이터 세트에 적용해 평가하는 프로세스**이다. <br>
학습, 검증, 테스트 데이터 세트로 나눌 수 있다. 

<img src = 'image/Cross Validation.jpg' alt = 'Cross Validation' width='700' height='700'>

**K 폴드 교차 검증**

K 폴드 교차 검증은 가장 보편적으로 사용되는 교차 검증 기법이다. <br>
**K개의 데이터 폴드 세트를 만들어서 K번만큼 각 폴드 세트에 학습과 검증 평가를 반복적으로 수행하는 방법**이다. 

<img src = 'image/KFold Cross Validation.jpg' alt = 'KFold Cross Validation' width='700' height='700'>

학습 데이터 세트와 검증 데이터 세트를 점진적으로 변경하면서 마지막 K번째까지 학습과 검증을 수행한다. <br>
**K개의 예측 평가를 구했으면 이를 평균내 K 폴드 평가 결과로 반영**한다. <br>
사이킷런에서는 K 폴드 교차 검증 프로세스를 구현하기 위해 **KFold**와 **StratifiedKFold** 클래스를 제공한다. 

In [8]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
import numpy as np

iris = load_iris()
features = iris.data
label = iris.target
dt_clf = DecisionTreeClassifier(random_state = 156)

# 5개의 폴드 세트로 분리하는 KFold 객체와 폴드 세트별 정확도를 담을 리스트 객체 생성
kfold = KFold(n_splits = 5)
cv_accuracy = []
print('붓꽃 데이터 세트 크기 : ', features.shape[0])

붓꽃 데이터 세트 크기 :  150


KFold 객체는 split()을 호출하면 학습용 / 검증용 데이터로 분할할 수 있는 Index를 반환한다. 

In [10]:
n_iter = 0

# KFold 객체의 split()을 호출하면 폴드별 학습용, 검증용 테스트의 row Index를 array로 반환
for train_index, test_index in kfold.split(features):
    # kfold.split()으로 반환된 Index를 이용해 학습용, 검증용 테스트 데이터 추출
    X_train, X_test = features[train_index], features[test_index]
    Y_train, Y_test = label[train_index], label[test_index]
    # 학습 및 예측
    dt_clf.fit(X_train, Y_train)
    pred = dt_clf.predict(X_test)
    n_iter += 1
    # 반복 시마다 정확도 측정
    accuracy = np.round(accuracy_score(Y_test, pred), 4)
    train_size = X_train.shape[0]
    test_size = X_test.shape[0]
    print('#{0} 교차 검증 정확도 : {1}, 학습 데이터 크기 : {2}, 검증 데이터 크기 : {3}'
          .format(n_iter, accuracy, train_size, test_size))
    print('#{0} 검증 세트 Index : {1}'.format(n_iter, test_index))
    cv_accuracy.append(accuracy)
    
# 개별 iteration별 정확도를 합하여 평균 정확도 계산
print('평균 검증 정확도 : ', np.mean(cv_accuracy))

#1 교차 검증 정확도 : 1.0, 학습 데이터 크기 : 120, 검증 데이터 크기 : 30
#1 검증 세트 Index : [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29]
#2 교차 검증 정확도 : 0.9667, 학습 데이터 크기 : 120, 검증 데이터 크기 : 30
#2 검증 세트 Index : [30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53
 54 55 56 57 58 59]
#3 교차 검증 정확도 : 0.8667, 학습 데이터 크기 : 120, 검증 데이터 크기 : 30
#3 검증 세트 Index : [60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83
 84 85 86 87 88 89]
#4 교차 검증 정확도 : 0.9333, 학습 데이터 크기 : 120, 검증 데이터 크기 : 30
#4 검증 세트 Index : [ 90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119]
#5 교차 검증 정확도 : 0.7333, 학습 데이터 크기 : 120, 검증 데이터 크기 : 30
#5 검증 세트 Index : [120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137
 138 139 140 141 142 143 144 145 146 147 148 149]
평균 검증 정확도 :  0.9


**Stratified K 폴드** <br>

Stratified K 폴드는 **불균형한 분포도를 가진 레이블 데이터 집합을 위한 K 폴드 방식**이다. <br>
불균형한 분포도를 가진 레이블 데이터 집합은 **특정 레이블 값이 특이하게 많거나 매우 적어서 값의 분포가 한쪽으로 치우치는 것**을 말한다. <br>
레이블 값으로 1이 특정 개별 반복별 학습 / 테스트 데이터 세트에는 상대적으로 많이 들어있고, 다른 반복 학습 / 테스트 데이터 세트에는 그렇지 못한 결과가 발생한다. <br>
우너본 데이터와 유사한 레이블 값의 분포를 학습 / 테스트 데이터 세트에도 유지하는 것이 매우 중요하다. <br>
**Stratified K 폴드는 원본 데이터의 레이블 분포를 먼저 고려한 뒤 이 분포와 동일하게 학습과 검증 데이터 세트를 분배**한다. 

In [11]:
import pandas as pd

iris = load_iris()
iris_df = pd.DataFrame(data = iris.data, columns = iris.feature_names)
iris_df['label'] = iris.target
iris_df['label'].value_counts()

0    50
1    50
2    50
Name: label, dtype: int64

In [14]:
kfold = KFold(n_splits = 3)
n_iter = 0 
for train_index, test_index in kfold.split(iris_df):
    n_iter += 1
    label_train = iris_df['label'].iloc[train_index]
    label_test = iris_df['label'].iloc[test_index]
    print('교차 검증 : {0}'.format(n_iter))
    print('학습 레이블 데이터 분포 : \n', label_train.value_counts())
    print('검증 레이블 데이터 분포 : \n', label_test.value_counts())

교차 검증 : 1
학습 레이블 데이터 분포 : 
 1    50
2    50
Name: label, dtype: int64
검증 레이블 데이터 분포 : 
 0    50
Name: label, dtype: int64
교차 검증 : 2
학습 레이블 데이터 분포 : 
 0    50
2    50
Name: label, dtype: int64
검증 레이블 데이터 분포 : 
 1    50
Name: label, dtype: int64
교차 검증 : 3
학습 레이블 데이터 분포 : 
 0    50
1    50
Name: label, dtype: int64
검증 레이블 데이터 분포 : 
 2    50
Name: label, dtype: int64


**StratifiedKFold는 레이블 데이터 분포도에 따라 학습 / 검증 데이터를 나누기 때문에 split() 메서드에 인자로 피처 데이터 세트뿐만 아니라 레이블 데이터 세트도 반드시 필요하다**는 점을 제외하면 StratifiedKFold를 사용하는 방법은 KFold를 사용하는 방법과 비슷하다. 

In [15]:
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits = 3)
n_iter = 0

for train_index, test_index in skf.split(iris_df, iris_df['label']):
    n_iter += 1
    label_train = iris_df['label'].iloc[train_index]
    label_test = iris_df['label'].iloc[test_index]
    print('교차 검증 : {0}'.format(n_iter))
    print('학습 레이블 데이터 분포 : \n', label_train.value_counts())
    print('검증 레이블 데이터 분포 : \n', label_test.value_counts())

교차 검증 : 1
학습 레이블 데이터 분포 : 
 2    34
0    33
1    33
Name: label, dtype: int64
검증 레이블 데이터 분포 : 
 0    17
1    17
2    16
Name: label, dtype: int64
교차 검증 : 2
학습 레이블 데이터 분포 : 
 1    34
0    33
2    33
Name: label, dtype: int64
검증 레이블 데이터 분포 : 
 0    17
2    17
1    16
Name: label, dtype: int64
교차 검증 : 3
학습 레이블 데이터 분포 : 
 0    34
1    33
2    33
Name: label, dtype: int64
검증 레이블 데이터 분포 : 
 1    17
2    17
0    16
Name: label, dtype: int64


출력 결과를 보면 학습 레이블과 검증 레이블 데이터 값의 분포도가 **거의 동일하게 할당** 되었음을 알 수 있다. 

In [23]:
df_clf = DecisionTreeClassifier(random_state = 156)

skfold = StratifiedKFold(n_splits = 3)
n_iter = 0
cv_accuracy = []

# StratifiedKFold의 split() 호출 시 반드시 레이블 데이터 세트도 추가 입력이 필요하다. 
for train_index, test_index in skfold.split(features, label):
    # split()으로 반환된 Index를 이용해 학습용, 검증용 테스트 데이터 추출
    X_train, X_test = features[train_index], features[test_index]
    Y_train, Y_test = label[train_index], label[test_index]
    
    # 학습 및 예측
    dt_clf.fit(X_train, Y_train)
    pred = dt_clf.predict(X_test)
    
    # 반복 시마다 정확도 측정
    n_iter += 1
    accuracy = np.round(accuracy_score(Y_test, pred), 4)
    train_size = X_train.shape[0]
    test_size = X_test.shape[0]
    print('#{0} 교차 검증 정확도 : {1}, 학습 데이터 크기 : {2}, 검증 데이터 크기 : {3}'
         .format(n_iter, accuracy, train_size, test_size))
    print('#{0} 검증 세트 인덱스 : {1}'.format(n_iter, test_index))
    cv_accuracy.append(accuracy)
    
# 교차 검증별 정확도 및 평균 정확도 계산
print('교차 검증별 정확도 : ', np.round(cv_accuracy, 4))
print('평균 검증 정확도 : ', np.round(cv_accuracy), 4)

#1 교차 검증 정확도 : 0.98, 학습 데이터 크기 : 100, 검증 데이터 크기 : 50
#1 검증 세트 인덱스 : [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  50
  51  52  53  54  55  56  57  58  59  60  61  62  63  64  65  66 100 101
 102 103 104 105 106 107 108 109 110 111 112 113 114 115]
#2 교차 검증 정확도 : 0.94, 학습 데이터 크기 : 100, 검증 데이터 크기 : 50
#2 검증 세트 인덱스 : [ 17  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  67
  68  69  70  71  72  73  74  75  76  77  78  79  80  81  82 116 117 118
 119 120 121 122 123 124 125 126 127 128 129 130 131 132]
#3 교차 검증 정확도 : 0.98, 학습 데이터 크기 : 100, 검증 데이터 크기 : 50
#3 검증 세트 인덱스 : [ 34  35  36  37  38  39  40  41  42  43  44  45  46  47  48  49  83  84
  85  86  87  88  89  90  91  92  93  94  95  96  97  98  99 133 134 135
 136 137 138 139 140 141 142 143 144 145 146 147 148 149]
교차 검증별 정확도 :  [0.98 0.94 0.98]
평균 검증 정확도 :  [1. 1. 1.] 4


Stratified K 폴드의 경우 원본 데이터의 레이블 분포도 특성을 반영한 학습 및 검증 데이터 세트를 만들 수 있으므로 **왜곡된 레이블 데이터 세트에서는 반드시 Stratified K 폴드를 이용해 교차 검증을 해야한다.** <br>
일반적으로 분류에서의 교차 검증은 K 폴드가 아니라 Stratified K 폴드로 분할되어야 한다. <br> 
**회귀에서는 Stratified K 폴드가 지원되지 않는다.** <br>
회귀 결정값은 이산값 형태의 레이블이 아니라 **연속된 숫자값이기 때문에** 결정값별로 분포를 정하는 의미가 없기 때문이다. 

**cross_val_score()** <br>

1. KFold로 데이터를 학습하고 예측하는 코드가 있을 때, 폴드 세트를 설정한다. <br>
2. for 루프에서 반복으로 학습 및 테스트 데이터의 Index를 추출한다. <br>
3. 반복적으로 학습과 예측을 수행하고 예측 성능을 반환한다. <br>

**cross_val_score()는 위 3단계 과정을 한번에 수행하는 API이다.** <br>

In [ ]:
cross_val_score(estimator, X, Y = None, scoring = None, cv = None, n_jobs = 1, verbose = 0, fit_params = None, 
               pre_dispatch = '2*n_jobs')

**estimator**는 사이킷런의 분류 알고리즘 클래스인 Classifier 또는 회귀 알고리즘 클래스인 Regressor를 의미한다. <br>
**X**는 피처 데이터 세트, **Y**는 레이블 데이터 세트, **scoring**은 예측 성능 평가 지표를 기술하며 **cv**는 교차 검증 폴드 수를 의미한다. <br>
cross_val_score() 수행 후 반환 값은 **scoring 파라미터로 지정된 성능 지표 측정값을 배열 형태로 반환**한다. <br>
cross_val_score()는 classifier가 입력되면 Stratified K 폴드 방식으로 레이블값의 분포에 따라 학습 / 테스트 세트를 분할한다. 

In [26]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score, cross_validate

iris_data = load_iris()
dt_clf = DecisionTreeClassifier(random_state = 156)

data = iris_data.data
label = iris_data.target

# 성능 지표는 정확도, 쿄차 검증 세트는 3개
scores = cross_val_score(df_clf, data, label, scoring = 'accuracy', cv = 3)
print('교차 검증별 정확도 : ', np.round(scores, 4))
print('평균 검증 정확도 : ', np.round(np.mean(scores), 4))

교차 검증별 정확도 :  [0.98 0.94 0.98]
평균 검증 정확도 :  0.9667


cross_val_score()는 **cv로 지정된 횟수만큼 scoring 파라미터로 지정된 평가 지표로 평가 결과값을 배열로 반환**한다. <br>
API는 내부에서 Estimator를 학습(fit), 예측(predict), 평가(evaluation)시켜주기 때문에 간단하게 교차 검증을 수행할 수 있다. <br>

비슷한 API로 cross_validate()가 있다. <br>
**cross_validate()** 는 **여러 개의 평가 지표를 반환**할 수 있다. <br>
학습 데이터에 대한 성능 평가 지표와 수행 시간도 같이 제공된다. 

### GridSearchCV - 교차 검증과 최적 하이퍼 파라미터 튜닝을 한번에 

사이킷런은 **GridSearchCV API**를 이용해 Classifier나 Regressor와 같은 알고리즘에 사용되는 **하이퍼 파라미터를 순차적으로 입력**하면서 편리하게 **최적의 파라미터를 도출**할 수 있는 방법을 제공한다. 

In [42]:
grid_parameters = {'max_depth' : [1, 2, 3],
                  'min_samples_split' : [2, 3]}

GridSearchCV는 교차 검증을 기반으로 하이퍼 파라미터의 최적 값을 찾는다. <br>
학습 / 테스트 세트로 자동 분할할 뒤 모든 파라미터를 순차적으로 적용하는 원리이다. <br>
동시에 순차적으로 파라미터를 테스트하므로 수행 시간이 상대적으로 오래 걸리는 것이 단점이다. <br>

**GridSearchCV 클래스의 생성자로 들어가는 주요 파라미터** <br>

|용어|설명|
|:------:|:---:|
|estimator|classifier, regressor, pipline이 사용될 수 있다.|
|param_grid|key + 리스트 값을 가지는 딕셔너리가 주어진다. estimator의 튜닝을 위해 파라미터 이름과 사용될 여러 파라미터 값을 지정한다.|
|scoring|예측 성능을 측정할 평가 방법을 지정한다. 보통은 사이킷런의 성능 평가 지표를 지정하는 문자열로 지정하거나 별도의 성능 평가 지표 함수도 지정할 수 있다.|
|cv|교차 검증을 위해 분할되는 학습 / 테스트 세트의 갯수를 지정한다.|
|refit|디폴트가 True로 생성 시 가장 최적의 하이퍼 파라미터를 찾은 뒤 입력된 estimator 객체를 해당 하이퍼 파라미터로 재학습시킨다.|

In [40]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

# 데이터를 로딩하고 학습 데이터와 테스트 데이터 분리
iris_data = load_iris()
X_train, X_test, Y_train, Y_test = train_test_split(iris_data.data, iris_data.target,
                                                   test_size = 0.2, random_state = 121)
dtree = DecisionTreeClassifier()

# 파라미터를 딕셔너리 형태로 설정
parameters = {'max_depth' : [1, 2, 3],
            'min_samples_split' : [2, 3]}

cv_results_는 GridSearchCV의 결과 세트로서 딕셔너리 형태로 key 값과 리스트 형태의 value 값을 가진다. 

In [41]:
import pandas as pd

# param_grid의 하이퍼 파라미터를 3개의 train, test set fold로 나누어 테스트 수행 설정
# refit = True가 디폴트이다. True이면 가장 좋은 파라미터 설정으로 재학습한다. 
grid_dtree = GridSearchCV(dtree, param_grid = parameters, cv = 3, refit = True)

# 붓꽃 학습 데이터로 param_grid의 하이퍼 파라미터를 순차적으로 학습 / 평가
grid_dtree.fit(X_train, Y_train)

# GridSearchCV 결과를 추출해 DataFrame으로 변환
scores_df = pd.DataFrame(grid_dtree.cv_results_)
scores_df[['params', 'mean_test_score', 'rank_test_score',
         'split0_test_score', 'split1_test_score', 'split2_test_score']]

,params,mean_test_score,rank_test_score,split0_test_score,split1_test_score,split2_test_score
0,"{'max_depth': 1, 'min_samples_split': 2}",0.700000,5,0.700,0.7,0.70
1,"{'max_depth': 1, 'min_samples_split': 3}",0.700000,5,0.700,0.7,0.70
2,"{'max_depth': 2, 'min_samples_split': 2}",0.958333,3,0.925,1.0,0.95
3,"{'max_depth': 2, 'min_samples_split': 3}",0.958333,3,0.925,1.0,0.95
4,"{'max_depth': 3, 'min_samples_split': 2}",0.975000,1,0.975,1.0,0.95
5,"{'max_depth': 3, 'min_samples_split': 3}",0.975000,1,0.975,1.0,0.95


위 결과에서 총 6개의 결과를 볼 수 있고, 이는 총 6번 변경하면서 학습 및 평가를 수행했음을 의미한다. <br>

**params column**는 수행할 때마다 적용된 개별 하이퍼 파라미터 값을 나타낸다. <br>
**rank_test_score**는 하이퍼 파라미터별로 성능이 좋은 score 순위를 나타낸다. 1이 가장 뛰어난 순위이며 이때의 파라미터가 최적의 하이퍼 파라미터이다. <br>
**mean_test_scroe**는 개별 하이퍼 파라미터별로 cv의 폴딩 테스트 세트에 대해 총 수행한 평가 평균값이다. <br>

GridSearchCV rorcpdml fit()을 수행하면 최고의 성능을 나타낸 하이퍼 파라미터의 값과 평가 결과 값이 각각 best_params_, best_score_ 속성에 기록된다. <br>

In [44]:
print('GridSearchCV 최적 파라미터 : ', grid_dtree.best_params_)
print('GridSearchCV 최고 정확도 : {0:.4f}'.format(grid_dtree.best_score_))

GridSearchCV 최적 파라미터 :  {'max_depth': 3, 'min_samples_split': 2}
GridSearchCV 최고 정확도 : 0.9750


GridSearchCV 객체의 생성 파라미터로 refit = True가 디폴트이다. <br>
**refit = True**이면 GridSearchCV가 **최적 성능을 나타내는 하이퍼 파라미터로 Estimator를 학습해 best_estimator_로 저장**한다.

In [46]:
# GridSearchCV의 refit으로 이미 학습된 estimator를 반환
estimator = grid_dtree.best_estimator_

# GridSearchCV의 best_estimator_는 이미 최적 학습이 됐으므로 별도 학습이 필요 없다.
pred = estimator.predict(X_test)
print('테스트 데이터 세트 정확도 : {0:.4f}'.format(accuracy_score(Y_test, pred)))

테스트 데이터 세트 정확도 : 0.9667


학습 데이터를 GridSearchCV를 이용해 최적 하이퍼 파라미터 튜닝을 수행한 뒤, 별도의 테스트 세트에서 이를 평가하는 것이 일반적인 머신러닝 모델 적용 방법이다. 